In [8]:
# make sure you have Github copilot installed, search it in the VSCode extension marketplace, it will make your coding much easier
import sys 
sys.path.append('/host/d/Github/')
import os
import torch
import numpy as np
import nibabel as nb
import CTDenoising_Diffusion_N2N.noise2noise.model as noise2noise
import CTDenoising_Diffusion_N2N.functions_collection as ff
import CTDenoising_Diffusion_N2N.Build_lists.Build_list as Build_list
import CTDenoising_Diffusion_N2N.noise2noise.Generator as Generator

# if it says no module named ... (e.g., lpips), do the following:
# 1. go to powershell, type wsl, enter wsl
# 2. in wsl, type: sudo docker container ls, you will see the container id of your running container
# 3. type: sudo docker container exec -it -u 0 <container_id> bash
# 4. now you are inside the container root, type: pip install lpips

### Step 1: define pre-trained model

In [ ]:
trial_name = 'noise2noise'
epoch = 78
# define your own saved model path and prediction save path
trained_model_filename = os.path.join('/host/d/projects/denoising/models', trial_name, 'models/model-' + str(epoch)+ '.pt')
save_folder = os.path.join('/host/d/projects/denoising/models', trial_name, 'pred_images'); os.makedirs(save_folder, exist_ok=True)

In [3]:
### parameters no need to change
image_size = [512,512]

histogram_equalization = True
background_cutoff = -1000
maximum_cutoff = 2000
normalize_factor = 'equation' 

### step 2: build patient list - testing, batch 5

In [ ]:
# example excel path (you can find it in this repo - example data folder)
# build_sheet =  Build_list.Build(os.path.join('/host/d/Github/CTDenoising_Diffusion_N2N/example_data/patient_lists/patient_list_supervised.xlsx'))
# use the spreadsheet I gave to you and put the path here:
build_sheet =  Build_list.Build(os.path.join('/host/d/projects/denoising/Patient_lists/fixedCT_static_simulation_train_test_gaussian_local.xlsx'))

_,patient_id_list,patient_subid_list,random_num_list, condition_list, x0_list = build_sheet.__build__(batch_list = [5])

### step 3: build model

In [6]:
# build model
model = noise2noise.Unet2D(
    init_dim = 16,
    channels = 2, 
    out_dim = 1,
    dim_mults = (2,4,8,16),
    full_attn = (None,None, False, True),
    act = 'ReLU',
)

in out is :  [(16, 32), (32, 64), (64, 128), (128, 256)]


### step 4: main

In [7]:
# load histogram equalization pre-saved files
# change to your own path, they are in this repo - example data folder
bins = np.load('/host/d/Github/CTDenoising_Diffusion_N2N/example_data/histogram_equalization/bins.npy') 
bins_mapped = np.load('/host/d/Github/CTDenoising_Diffusion_N2N/example_data/histogram_equalization/bins_mapped.npy')

In [18]:
# main
for i in range(0, 1):#patient_id_list.shape[0]):
    patient_id = patient_id_list[i]
    patient_subid = patient_subid_list[i]
    random_num = random_num_list[i]
    x0_file = x0_list[i]
    condition_file = condition_list[i]

    print(i,patient_id, patient_subid, random_num)

    # # get the ground truth image
    # gt_img = nb.load(x0_file)
    # affine = gt_img.affine; gt_img = gt_img.get_fdata()[:,:,30:80]

    # get the condition image
    condition_img = nb.load(condition_file).get_fdata()[:,:,30:80]
    affine = nb.load(condition_file).affine
    

    # make folders
    ff.make_folder([os.path.join(save_folder, patient_id), os.path.join(save_folder, patient_id, patient_subid), os.path.join(save_folder, patient_id, patient_subid, 'random_' + str(random_num))])
    save_folder_case = os.path.join(save_folder, patient_id, patient_subid, 'random_' + str(random_num), 'epoch' + str(epoch)); os.makedirs(save_folder_case, exist_ok=True)

    # save condition image
    nb.save(nb.Nifti1Image(condition_img, affine), os.path.join(save_folder_case,'condition_img.nii.gz'))

    # # generator
    generator = Generator.Dataset_2D(
        img_list = np.array([condition_file]),
        image_size = image_size,

        num_slices_per_image = 50,
        random_pick_slice = False,
        slice_range = [30,80],

        bins = bins,
        bins_mapped = bins_mapped,
        histogram_equalization = histogram_equalization,
        background_cutoff = background_cutoff,
        maximum_cutoff = maximum_cutoff,
        normalize_factor = normalize_factor,)

    # # sample:
    sampler = noise2noise.Sampler(model,generator,batch_size = 1, image_size = image_size)

    pred_img = sampler.sample_2D(trained_model_filename, condition_img)
    pred_img_final = pred_img
    
    # save
    nb.save(nb.Nifti1Image(pred_img_final, affine), os.path.join(save_folder_case, 'pred_img.nii.gz'))

0 00004038 0000455420 0
histogram equalization:  True
model device:  cuda:0
reference_img shape:  (512, 512, 50)
